In [1]:
import numpy as np
import sympy as sp
from sympy import ntheory
from sympy.ntheory import continued_fraction_periodic
import math
from itertools import chain

In [2]:
def classRepresentatives(trace): # gives quadratic forms of given trace
    bound = math.floor(trace**2-4)
    quadraticForms = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            quadraticForms+=representatives(D)
        elif D%4 == 1:
            quadraticForms+=representatives(D)
        i = i + 1
    return [Q for Q in quadraticForms if tr(Q)==np.round(trace)]

def representatives(D):
    reps = []
    seenFracs = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            if frac not in seenFracs and gcd(A,B,C)==1:
                seenFracs+=permutationsOf(frac)
                reps.append(Q)
            else:
                continue
    return reps

def permutationsOf(a):
    perms = []
    if len(a)%2==0:
        for n in range(0,len(a),2):
            perms.append(a[n:]+a[:n])
    else:
        for n in range(0,len(a)):
            perms.append(a[n:]+a[:n])
    return perms

def disc(Q):
    return Q[1]**2-4*Q[0]*Q[2]

def areEquivalent(Q,W):
    return ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0] in permutationsOf(ntheory.continued_fraction_periodic(-W[1], 2*W[0], disc(W))[0])

def equalsQuad(L,M):
    if len(L)!=len(M):
        return False
    for l in L:
        counter = 0
        for m in M:
            if areEquivalent(m,l):
                M.remove(m)
                counter+=1
        if counter !=1:
            return False
    return True

def classNum(D):
    return len(representatives(D))

def tr(Q):
    return np.trace(quadToMat(Q))

def isASquare(n):
    for i in range(2,n):
        if n==i**2:
            return True
    return False

def gcd(a,b,c):
    return np.gcd(a,np.gcd(b,c))

def matgcd(mat):
    return gcd(mat[1,0],mat[1,1]-mat[0,0],mat[0,1])

def quadToMat(Q):
    mat = np.identity(2)
    length = 0
    for x in ntheory.continued_fraction_periodic(-Q[1], 2*Q[0],disc(Q))[0]:
        mat = np.matmul(mat,[[x,1],[1,0]])
        length += 1
    if length % 2 == 1:
        mat = np.matmul(mat,mat)
    return np.array(mat).astype(int)

In [185]:
#REMEMBER: 0 <-> L AND 1 <-> R
def contFracToLR(contFrac): #sends a continued fraction [a0,a1,...,an] to LLRR.. (L's on even, R's on odd, as many as said)
    if len(contFrac)%2==1:
        contFrac = contFrac + contFrac #if length is odd, then double it
    LRsequence = []
    for i in range(len(contFrac)):
        if i%2==1: #if i is odd, i.e. at an odd index in the continued fraction
            for j in range(contFrac[i]): #then look at how many there are indicated, and append that many 1's
                LRsequence.append(1)
        else:
            for j in range(contFrac[i]): #if i is even, i.e. at even index, look at how many are indicated, and append 0's
                LRsequence.append(0)
    return LRsequence

def recover2(ls): #sends an LR sequence to a cont'd fraction (not clear whether this is needed)
    output = []
    i=0
    while i<len(ls):
        counter=1
        while(i<len(ls)-1 and ls[i] == ls[i+1]):
            i+=1
            counter+=1
        i+=1
        output.append(counter)
    if (len(output)%2!=0):
        output[len(output)-1]+=output[0]
        output = output[1:]
    return output
def inv(lrSeq): #inverts a list of 0's and 1's (R's and L's)
    return list((np.array(lrSeq[::-1])-1)**2)

def indices(lrSeq): #this outputs all of the positions of R's (R=1) (i.e. indices where there is an R)
    return [i for i in range(len(lrSeq)) if lrSeq[i]==1]

def pairs(lrSeq): #this outputs all pairs of indices of two R's
    output = []
    for i in indices(lrSeq):
        for j in indices(lrSeq):
            if i==j:
                continue
            else:
                output.append((i,j))
    return output

def pairToFrac(s1,s2): #sends a pair of LR sequences to continued fraction s_1 R s_2^-1 R
    out = s1+[1]+inv(s2)+[1]
    return recover2(out)

def split2(contFrac): #splits a cont'd fraction on every possible pair of R's into 2 groups of R's and L's (in 0's and 1's)
    output = []
    converted = contFracToLR(contFrac)
    n = len(converted)
    splitPoints = pairs(converted)
    for i,j in splitPoints:
        temp = converted[i:]+converted[:i]
        temp1 = temp[1:(j-i)%n]
        temp2 = inv(temp[(j-i)%n+1:]) #inv(temp[(j-i)%n+1:])
        output.append([temp1,temp2])
    return output

def getSplitList(cfList): #input a list of continued fractions, and get list of split decompositions for each fraction
    listOfSplits = []
    for frac in cfList:
        listOfSplits.append(split2(frac))
    return listOfSplits

def getThetaGraph(cfList): #input list of continued fractions, and output triple of compatible fractions
    tripleOfSplits = []
    listOfThetaGraphs = [] #empty list to put theta structures in
    listOfSplits = getSplitList(cfList) #list of list of splits for each continued fraction
    numberOfContinuedFractions = len(cfList) #the number of continued fractions
    for i in range(numberOfContinuedFractions): #loop over each continued fraction
        compareAgainstInput = [] #empty list to put other split elements in
        
        for j in chain(range(i), range(i+1,numberOfContinuedFractions)): #loop to consider all LR sequences that are not the i-th continued fraction
            for splits in listOfSplits[j]:
                compareAgainstInput.append(splits)
        for pair in listOfSplits[i]: #loop over every LR sequence in the i-th continued fraction
            candidatesForPair = [] #empty list to put other elements that are compatible with pair
            
            for obj in compareAgainstInput: #loop over compatible elements
                if obj[0] == inv(pair[1]): #if first sequence is inverse of second element of pair, put it in list
                    candidatesForPair.append(obj)
                if obj[1] == inv(pair[0]): #if second sequence is inverse of first element of pair, put it in list
                    candidatesForPair.append(obj)
                    
            for x in candidatesForPair: #loop over all tuples in candidates
                
                for y in candidatesForPair:
                    if ((x[1] == inv(y[0]))): #if these are compatible, they form a Theta graph. the bottom just stops permutations
                        if ([pairToFrac(pair[0],pair[1]),pairToFrac(x[0],x[1]),pairToFrac(y[0],y[1])] not in listOfThetaGraphs) and ([pairToFrac(pair[0],pair[1]),pairToFrac(y[0],y[1]),pairToFrac(x[0],x[1])] not in listOfThetaGraphs) and ([pairToFrac(y[0],y[1]),pairToFrac(pair[0],pair[1]),pairToFrac(x[0],x[1])] not in listOfThetaGraphs) and ([pairToFrac(y[0],y[1]),pairToFrac(x[0],x[1]),pairToFrac(pair[0],pair[1])] not in listOfThetaGraphs) and ([pairToFrac(x[0],x[1]),pairToFrac(pair[0],pair[1]),pairToFrac(y[0],y[1])] not in listOfThetaGraphs) and ([pairToFrac(x[0],x[1]),pairToFrac(y[0],y[1]),pairToFrac(pair[0],pair[1])] not in listOfThetaGraphs):
                            tripleOfSplits.append([pair,x,y])
                            listOfThetaGraphs.append([pairToFrac(pair[0],pair[1]),pairToFrac(x[0],x[1]),pairToFrac(y[0],y[1])]) #add to list of theta graphs
                            
    return [tripleOfSplits,listOfThetaGraphs]

In [184]:
getSplitList([[1,1,2,2,2,1],[1,1,3,3],[1,8]])

[[[[0, 0], [1, 0, 1, 1, 0]],
  [[0, 0, 1], [1, 0, 1, 1]],
  [[0, 0, 1, 1, 0, 0], [1]],
  [[1, 0, 0, 1, 0], [1, 1]],
  [[], [1, 1, 0, 1, 0, 1, 1]],
  [[1, 0, 0], [1, 1, 0, 1]],
  [[0, 0, 1, 0], [0, 1, 1]],
  [[0, 0, 1, 0, 1, 0, 0], []],
  [[0, 0], [0, 1, 1, 0, 1]],
  [[0], [1, 1, 0, 0, 1, 1]],
  [[0, 1, 0, 0], [1, 1, 0]],
  [[0, 1, 0, 0, 1], [1, 1]]],
 [[[0, 0, 0], [1, 0, 0]],
  [[0, 0, 0, 1], [1, 0]],
  [[0, 0, 0, 1, 1], [1]],
  [[1, 1, 0], [1, 1, 1]],
  [[], [1, 1, 1, 0, 1, 0]],
  [[1], [1, 1, 1, 0, 1]],
  [[1, 0], [0, 1, 1, 1]],
  [[1, 0, 1, 0, 0, 0], []],
  [[], [0, 1, 1, 1, 0, 1]],
  [[0], [0, 0, 1, 1, 1]],
  [[0, 1, 0, 0, 0], [0]],
  [[0, 1, 0, 0, 0, 1], []]],
 [[[], [1, 0, 0, 0, 0, 0, 0]],
  [[1], [1, 0, 0, 0, 0, 0]],
  [[1, 1], [1, 0, 0, 0, 0]],
  [[1, 1, 1], [1, 0, 0, 0]],
  [[1, 1, 1, 1], [1, 0, 0]],
  [[1, 1, 1, 1, 1], [1, 0]],
  [[1, 1, 1, 1, 1, 1], [1]],
  [[1, 1, 1, 1, 1, 1, 0], []],
  [[], [0, 1, 0, 0, 0, 0, 0]],
  [[1], [0, 1, 0, 0, 0, 0]],
  [[1, 1], [0, 1, 0, 0, 0]],
 

In [183]:
candidates([[1,1,2,2,2,1],[1,1,3,3],[1,8]])

[[0, 0], [1, 0, 1, 1, 0]] []
[[0, 0, 1], [1, 0, 1, 1]] []
[[0, 0, 1, 1, 0, 0], [1]] [[[0], [0, 0, 1, 1, 1]], [[0], [0, 0, 0, 0, 0, 0]]]
[[1, 0, 0, 1, 0], [1, 1]] []
[[], [1, 1, 0, 1, 0, 1, 1]] [[[1, 0, 1, 0, 0, 0], []], [[0, 1, 0, 0, 0, 1], []], [[1, 1, 1, 1, 1, 1, 0], []], [[1, 1, 1, 1, 1, 0, 1], []], [[1, 1, 1, 1, 0, 1, 1], []], [[1, 1, 1, 0, 1, 1, 1], []], [[1, 1, 0, 1, 1, 1, 1], []], [[1, 0, 1, 1, 1, 1, 1], []], [[0, 1, 1, 1, 1, 1, 1], []]]
[[1, 0, 0], [1, 1, 0, 1]] []
[[0, 0, 1, 0], [0, 1, 1]] []
[[0, 0, 1, 0, 1, 0, 0], []] [[[], [1, 1, 1, 0, 1, 0]], [[], [0, 1, 1, 1, 0, 1]], [[], [1, 0, 0, 0, 0, 0, 0]], [[], [0, 1, 0, 0, 0, 0, 0]], [[], [0, 0, 1, 0, 0, 0, 0]], [[], [0, 0, 0, 1, 0, 0, 0]], [[], [0, 0, 0, 0, 1, 0, 0]], [[], [0, 0, 0, 0, 0, 1, 0]], [[], [0, 0, 0, 0, 0, 0, 1]]]
[[0, 0], [0, 1, 1, 0, 1]] []
[[0], [1, 1, 0, 0, 1, 1]] [[[0, 0, 0, 1, 1], [1]], [[1, 1, 1, 1, 1, 1], [1]]]
[[0, 1, 0, 0], [1, 1, 0]] []
[[0, 1, 0, 0, 1], [1, 1]] []
[[0, 0, 0], [1, 0, 0]] [[[1, 1, 0], [0, 0, 0